In [ ]:
import os
import numpy as np

In [ ]:
#test_file = 'dev.1_all.hdf5_base.npz'
#test_file = 'test.1_all.hdf5_base.npz'
test_file = 'test.1_all.hdf5_fac1.0.npz'
test_path = './orig/omerlevy-bidaf_no_answer-2e9868b224e4/relation_splits/'

data = np.load( os.path.join(test_path,test_file) )
data.files  # ['predictions', 'targets']

In [ ]:
preds, targs = data['predictions'], data['targets']
preds.shape, targs.shape  # ((10, 1, 5, 128), (10, 1, 128))

In [ ]:
def assess(pred, targ, debug=False):
    #print(pred.shape, targ.shape)  # (5, 128) (128,)
    
    #for i,v in enumerate(list(targ)):
    #    if v==0: continue
    #    print(i, v)  # 1->2 is subject, 3->4 is required answer(s)
        
    #  Get the starts and ends from targ
    targ_q_starts = [i for i,v in enumerate(list(targ)) if v==1]
    targ_a_starts = [i for i,v in enumerate(list(targ)) if v==3]
    #print(targ_ans_starts)
    
    targ_q_arr, targ_a_arr = [],[]
    for i in targ_q_starts:                    # find next '2'
        # problem at idx=1218 ??
        if 2 in list(targ[i:]):
            targ_q_arr.append( (i, i+list(targ[i:]).index(2) ) )
    for i in targ_a_starts:                    # find next '4'
        # problem at idx=4480 ??
        if 4 in list(targ[i:]):
            targ_a_arr.append( (i, i+list(targ[i:]).index(4) ) )
    print( 'targets : q=', targ_q_arr, ', a=', targ_a_arr )
    
    # This is pretty 'audacious', since it wasn't trained to succeed at doing this
    #print("                      answer: ", np.argmax(pred[3,:]), np.argmax(pred[4,:]),  )
    
    """
    Evaluation Metrics : Each instance is evaluated by comparing 
    the tokens in the labeled answer set with those of the predicted span. 
    Precision is the true positive count divided by the number of times the system returned a non-null answer. 
    Recall is the true positive count divided by the number of instances that have an answer.
    """
    """  NOT HANDLED
    We ignore word order, case, punctuation, and articles (“a”, “an”, “the”). 
    We also ignore “and”, which often appears when a single span captures multiple correct answers
    (e.g. “United States and Canada”).    
    """
    
    have_ans = 1 if len(targ_a_arr)>0 else 0
    
    tp=0.
    non_null=0

    a_start_best = a_end_best = 0
    valid=False    
    
    # Ok, so how about looking at the list of predicted classes?
    pred_argmax = np.argmax(pred[:,:], axis=0)
    #print(pred_argmax)
    pred_argmax_list = list(pred_argmax)
    if 3 in pred_argmax_list and 4 in pred_argmax_list:
        print(" - Found ideal")
        a_start_best = pred_argmax_list.index(3)
        a_end_best = pred_argmax_list.index(4)
        # This is a great method : good accuracy *if* it returns results
        valid =  a_start_best<a_end_best

    print(" - pred_argmax_start/end = ", 3 in pred_argmax_list, 4 in pred_argmax_list)
        
    # We didn't get any answers : Let's see if the best guesses are valid enough
    #if (3 not in pred_argmax_list) or (4 not in pred_argmax_list):
    #if (not valid) and (3 in pred_argmax_list or 4 in pred_argmax_list):
    if False:
        # Let's see if this naive method works (at all)
        #a_start_best = np.argmax(pred[3,:])
        #a_end_best   = np.argmax(pred[4,:])
        #
        #if pred[3, a_start_best]>pred[4, a_end_best]:
        #    a_end_best   = np.argmax(pred[4,a_start_best:])+a_start_best
        #else:
        #    a_start_best = np.argmax(pred[3,:a_end_best])
        
        # Go from the best start onwards
        a_start_0 = np.argmax(pred[3,:])
        a_end_0   = np.argmax(pred[4,a_start_0:])+a_start_0
                
        # Go from the best end backwards
        a_end_1   = np.argmax(pred[4,:])
        a_start_1 = np.argmax(pred[3,:a_end_1])
        
        if pred[3,a_start_0]+pred[4,a_end_0] > pred[3,a_start_1]+pred[4,a_end_1]:
            a_start_best, a_end_best = a_start_0, a_end_0
        else:
            a_start_best, a_end_best = a_start_1, a_end_1
        
        valid =  a_start_best<a_end_best

        if pred[3, a_start_best]<+0.0:
            valid=False
        if pred[4, a_end_best]<+0.0:
            valid=False
            
        print(" - Trying naive : ", a_start_best, a_end_best, 
              pred[3, a_start_best], pred[4, a_end_best])
            
        #valid=False

    if valid: # This appears to be a valid guess
        print("                      answer: [%d, %d]" % (a_start_best, a_end_best))
        non_null=1 # The system thinks it found something
        
        # Work out the overlap between (a_start_best,a_end_best) and (targ_a_arr)
        #targ_mask = np.zeros(targ.shape)
        #for (i,j) in targ_a_arr:
        #    for k in range(i,j+1):
        #        targ_mask[k]=1
        #targ
        
        # Work out the overlap between (a_start_best,a_end_best) and each element of (targ_a_arr)
        best_overlap=0.
        a0, a1 = a_start_best, a_end_best
        for (b0, b1) in targ_a_arr:
            intersection = min(a1, b1) - max(a0, b0)
            union        = max(a1, b1) - min(a0, b0)
            overlap_fraction = intersection/float(union)
            
            if overlap_fraction>0. and best_overlap<overlap_fraction:
                best_overlap=overlap_fraction
        tp = best_overlap
        
    else:  # Could we have guessed some other way?
        print("                      answer: []")
    

    if True and debug:
        for i,v in enumerate(list(pred.T)):
            print("%3d" % i, ["%+7.2f" % f for f in list(v)])
            
    if True and debug:
        for i,v in enumerate(list(pred.T)):
            c = np.argmax(v)
            if c==0: continue
            print(i, c)  # 1->2 is subject, 3->4 is required answer(s)

    
    return tp, non_null, have_ans

# idx=0 should be a None, idx=1 should have an Answer, 6 seems correct
idx=253; assess( preds[idx, 0, :, :], targs[idx, 0, :], debug=True)

In [ ]:
tp_tot, non_null_tot, have_ans_tot = 0.,0,0
for idx in range(preds.shape[0]):
    #print("idx=", idx)
    tp, non_null, have_ans = assess( preds[idx, 0, :, :], targs[idx, 0, :] )
    print("       #%d assessment :tp=%.1f, non_null=%1d, have_ans=%1d" % (idx, tp, non_null, have_ans,) )
    tp_tot += tp
    non_null_tot += non_null
    have_ans_tot += have_ans

precis = tp_tot / non_null_tot
recall = tp_tot / have_ans_tot

f1 = 2. * (precis*recall)/(precis+recall)

print("precision=%.2f%% recall=%.2f%% F1=%.2f%%" % (100.*precis, 100.*recall, 100.*f1))

In [ ]:
#Paper (BEAT THIS!)
# precision=43.61% recall=36.45% F1=39.61%

#test.1_all.hdf5_base.npz 
# precision=30.97% recall=33.06% F1=31.98%  # No restriction on start,end logit values
# precision=43.03% recall=23.35% F1=30.27%  # Force start,end logit values >0

#test.1_all.hdf5_fac1.0.npz 
# precision=24.67% recall=43.13% F1=31.39%  # No restriction on start,end logit values
# precision=29.29% recall=40.50% F1=34.00%  # Force start,end logit values >0
